In [2]:
import imutils
import numpy as np
import cv2
from PIL import Image
from IPython.display import display, Javascript

# Face Segmentation

In [6]:
#!wget -N https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt#
#!wget -N https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

In [3]:
print("[INFO] loading model...")
prototxt = 'deploy.prototxt'
model = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model)

[INFO] loading model...


In [4]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    (h, w) = img.shape[:2]

    #image = imutils.resize((np.reshape(np.frombuffer(b64decode(frame_js["img"].split(',')[1])),np.uint8),(640,480)),width=400)
    #print(frame_js["img"].split(','))
    #print(len(image))
    #image = (np.frombuffer(b64decode(frame_js["img"].split(',')[1]),np.uint8))
    #print(len(image))
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    for i in range(0, detections.shape[2]):
      # extract the confidence (i.e., probability) associated with the prediction
      confidence = detections[0, 0, i, 2]

      # filter out weak detections by ensuring the `confidence` is
      # greater than the minimum confidence threshold
      if confidence > 0.5:
        # compute the (x, y)-coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        # draw the bounding box of the face along with the associated probability
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        img = img[startY:endY,startX:endX]
        cv2.rectangle(img, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(img, text, (startX, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)    
    cv2.imshow('frame', img)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Face Recognition

In [5]:
!pip3 install deepface

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 1.5 MB/s eta 0:00:001.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.9 MB/s eta 0:00:00m eta 0:00:010:01:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 7.6 MB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 15.7 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 11.6 MB/s eta 0:00:00
     

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 14.0 MB/s eta 0:00:0031m14.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 39.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=14c568eb8c3dc5442f12692bb2e81e33d45c967c477fc442fab36ec52aeb4fc1
  Stored in directory: /home/loe/.cache/pip/wheels/a5/e5/b1/36f99d92494185177f07f79579ae168cb034574e98e9b16ae5
  Created wheel for gdown: filename=gdown-4.5.3-py3-none-any.whl size=14821 sha256=cee16af5923cbcb11006e98b3b7224ee3a124316d55c40158217f5ff05b9488d
  Stored in directory: /home/loe/.cache/pip/wheels/6c/dd/fe/4982475d227ab282a93ee540130e88cd43bb1698131b848d25
Successfully built fire gdown


In [7]:
from deepface import DeepFace


Directory  /home/loe /.deepface created
Directory  /home/loe /.deepface/weights created


In [8]:
verification = DeepFace.verify(img1_path = "img1.jpg", img2_path = "img2.jpg")

vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /home/loe/.deepface/weights/vgg_face_weights.h5
100%|██████| 580M/580M [01:43<00:00, 5.59MB/s]


ValueError: ('Confirm that ', 'img1.jpg', ' exists')

In [11]:
#DeepFace.stream(db_path = “db”)
DeepFace.stream(db_path="db")
#Read more at: https://viso.ai/computer-vision/deepface/

Detector backend is  opencv
facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /home/loe/.deepface/weights/facial_expression_model_weights.h5
100%|████| 5.98M/5.98M [00:01<00:00, 4.02MB/s]


Emotion model loaded
age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /home/loe/.deepface/weights/age_model_weights.h5
100%|██████| 539M/539M [01:09<00:00, 7.79MB/s]


Age model loaded
gender_model_weights.h5 will be downloaded...


...

Gender model loaded
Facial attibute analysis models loaded in  160.01853275299072  seconds


Finding embeddings: 0it [00:00, ?it/s]

Embeddings found for given data set in  0.004182338714599609  seconds


1/1 [==============================] - 0s 241ms/step


KeyboardInterrupt: 